In [1]:
import os
os.chdir("..")
print(os.getcwd())

/data/akhlak/PromptDataExtraction


In [19]:
import importlib

import warnings
warnings.simplefilter('ignore')

In [3]:
import sett
import pylogg

sett.load_settings()
pylogg.setLevel(pylogg.DEBUG)

Load OK: settings.yaml


In [4]:
import json
from backend.nlp import ner

def BERT():
    """ Load the MaterialsBERT model and return the model handle. """
    bertModel = ner.MaterialsBERT(sett.NerModel.model)
    bertModel.init_local_model(device=0)
    return bertModel

def Tg_dataset():
    """ Return a list of (doi, text) tuples from Tg ground dataset. """
    datajson = "tests/tg_ground_data.json"
    with open(datajson) as fp:
        data = json.load(fp)
    abstracts = []
    for k, v in data.items():
        for item in v:
            if item['abstract']:
                abstracts.append((k, item['abstract']))
    return abstracts

bert = BERT()
dataset = Tg_dataset()

Using /home/akhlak/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/akhlak/.cache/torch_extensions/py310_cu117/cuda_kernel/build.ninja...
Building extension module cuda_kernel...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
      -- ner_ Loaded materials bert.


Loading extension module cuda_kernel...


In [6]:
from backend.text import normalize
from backend import ner_pipeline

In [7]:
for i, (doi, text) in enumerate(dataset):
    text = normalize.normText(text)
    tags = bert.get_tags(text)
    extractor = ner_pipeline.TextDataExtractor(text, tags)
    mat = extractor.extract(debug=True)
    mat._detect_polymer_type()
    # mat._coreference_material_entities()

    if i >= 10:
        break


  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.
  --  -- materials_ Detect polymer types.


In [38]:
dataset[i]

('10.1080/10236661003746405',
 'In this study, the mechanical properties and glass transition temperature of PLA-PEG block copolymer were investigated by molecular dynamic simulation. The software package Discover in Materials Studio with condensed-phase optimized molecular potentials for atomistic simulation studies (COMPASS) force field was used for the simulation. To verify the validity of simulation results, PLA and PLA-PEG copolymer were prepared in this research and the glass transition temperature was measured by DSC. The simulation results showed that the random combination chain has more probability than the alternating combination chain to exist in copolymer structure. Furthermore, PLA-PEG block copolymer has better tensile strength but lower shear strength than PLA, which agree well with the theoretical results. The simulation values of T_{g} (PLA: 333.97 K and PLA-PEG: 326.70 K) agree well with the DSC results (PLA: 328.8 K and PLA-PEG: 321.3 K). The conclusion could be dra

In [95]:
from backend.nlp import materials
from backend.nlp import ner
from backend.text import normalize

importlib.reload(materials)
importlib.reload(ner)
importlib.reload(normalize)

<module 'backend.text.normalize' from '/data/akhlak/PromptDataExtraction/backend/text/normalize.py'>

In [96]:
extractor = ner_pipeline.TextDataExtractor(text, tags)
mat = extractor.extract(debug=True)
mat._detect_polymer_type()

  --  -- materials_ Detect polymer types.


In [97]:
" ".join([t.text for t in mat.grouped_spans])

'In this study, the mechanical properties and glass transition temperature of PLA-PEG block copolymer were investigated by molecular dynamic simulation. The software package Discover in Materials Studio with condensed-phase optimized molecular potentials for atomistic simulation studies (COMPASS) force field was used for the simulation. To verify the validity of simulation results, PLA and PLA-PEG copolymer were prepared in this research and the glass transition temperature was measured by DSC. The simulation results showed that the random combination chain has more probability than the alternating combination chain to exist in copolymer structure. Furthermore, PLA-PEG block copolymer has better tensile strength but lower shear strength than PLA , which agree well with the theoretical results. The simulation values of T_{g} ( PLA : 333.97K and PLA-PEG : 326.70K ) agree well with the DSC results ( PLA : 328.8K and PLA-PEG : 321.3K ). The conclusion could be drawn from the results that m

In [99]:
mat.grouped_spans

[NerLabelGroup(text='In this study, the mechanical properties and glass transition temperature of', label='O', start=0, end=11),
 NerLabelGroup(text='PLA-PEG', label='POLYMER', start=12, end=14),
 NerLabelGroup(text='block copolymer were investigated by molecular dynamic simulation. The software package Discover in Materials Studio with condensed-phase optimized molecular potentials for atomistic simulation studies (COMPASS) force field was used for the simulation. To verify the validity of simulation results,', label='O', start=15, end=60),
 NerLabelGroup(text='PLA', label='POLYMER', start=61, end=61),
 NerLabelGroup(text='and', label='O', start=62, end=62),
 NerLabelGroup(text='PLA-PEG', label='POLYMER', start=63, end=65),
 NerLabelGroup(text='copolymer were prepared in this research and the glass transition temperature was measured by DSC. The simulation results showed that the random combination chain has more probability than the alternating combination chain to exist in copolymer

In [100]:
mat.material_mentions

[Polymer(name='PLA-PEG', tag='POLYMER', coreferents=['PLA-PEG'], is_homopolymer=False, is_copolymer=True, is_starpolymer=False),
 Polymer(name='PLA', tag='POLYMER', coreferents=['PLA'], is_homopolymer=True, is_copolymer=False, is_starpolymer=False),
 Polymer(name='PLA-PEG', tag='POLYMER', coreferents=['PLA-PEG'], is_homopolymer=False, is_copolymer=True, is_starpolymer=False),
 Polymer(name='PLA-PEG', tag='POLYMER', coreferents=['PLA-PEG'], is_homopolymer=False, is_copolymer=True, is_starpolymer=False),
 Polymer(name='PLA', tag='POLYMER', coreferents=['PLA'], is_homopolymer=True, is_copolymer=False, is_starpolymer=False),
 Polymer(name='PLA', tag='POLYMER', coreferents=['PLA'], is_homopolymer=True, is_copolymer=False, is_starpolymer=False),
 Polymer(name='PLA-PEG', tag='POLYMER', coreferents=['PLA-PEG'], is_homopolymer=False, is_copolymer=True, is_starpolymer=False),
 Polymer(name='PLA', tag='POLYMER', coreferents=['PLA'], is_homopolymer=True, is_copolymer=False, is_starpolymer=False),


In [89]:
mat.abbreviation_pairs.append(('PLA-PEG', 'PEG'))
mat.abbreviation_pairs.append(('PLA', 'Polylactic Acid'))
mat.abbreviation_pairs.append(('PLA', 'PLA-PEG'))
mat.abbreviation_pairs.append(('PLA', 'PEG'))

In [101]:
mat._detect_materials_coreferents()

  --  -- materials_ Detect polymer correferents.


[Polymer(name='PLA-PEG', tag='POLYMER', coreferents=['PLA-PEG', 'pla-peg'], is_homopolymer=False, is_copolymer=True, is_starpolymer=False),
 Polymer(name='PLA', tag='POLYMER', coreferents=['PLA', 'pla'], is_homopolymer=True, is_copolymer=False, is_starpolymer=False)]